In [1]:
import re
import pandas as pd
import numpy as np

# Lipid Datasets

MS DIAL lipidome atlas + Swiss lipids

### Swiss Lipids

In [2]:
swiss_lipids = pd.read_csv('swiss_lipids.csv')
swiss_lipids.head(5)

/var/folders/q1/mfxy01nj293b7bbnnhjkt_6w0000gn/T/ipykernel_6626/399708200.py:1: DtypeWarning: Columns (4,6,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  swiss_lipids = pd.read_csv('swiss_lipids.csv')


,mcid,Level,Name,Abbreviation*,Synonyms*,Lipid class*,Parent,Components*,smiles,InChI (pH7.3),...,Exact m/z of [M+Li]+,Exact m/z of [M+NH4]+,Exact m/z of [M-H]-,Exact m/z of [M+Cl]-,Exact m/z of [M+OAc]-,CHEBI,LIPID MAPS,HMDB,MetaNetX,PMID
0,SLM:000000002,Class,Ceramide (iso-d17:1(4E)),Cer(iso-d17:1(4E)),N-acyl-15-methylhexadecasphing-4-enine,SLM:000399814,NaN,NaN,CC(C)CCCCCCCCC\C=C\[C@@H](O)[C@H](CO)NC([*])=O,InChI=none,...,NaN,NaN,NaN,NaN,NaN,70846,NaN,NaN,MNXM97012,| 11443131 | 14685263 | 18390550 | 21325339 |...
1,SLM:000000003,Isomeric subspecies,15-methylhexadecasphing-4-enine,NaN,NaN,SLM:000390097,NaN,NaN,CC(C)CCCCCCCCC\C=C\[C@@H](O)[C@@H]([NH3+])CO,InChI=1S/C17H35NO2/c1-15(2)12-10-8-6-4-3-5-7-9...,...,292.282235,303.300605,284.259503,320.236181,344.280632,70771,NaN,NaN,MNXM57784,19372430
2,SLM:000000006,Isomeric subspecies,15-methylhexadecasphinganine,NaN,NaN,SLM:000390097,NaN,NaN,CC(C)CCCCCCCCCCC[C@@H](O)[C@@H]([NH3+])CO,InChI=1S/C17H37NO2/c1-15(2)12-10-8-6-4-3-5-7-9...,...,294.297885,305.316255,286.275153,322.251831,346.296282,70829,NaN,NaN,MNXM97029,19372430
3,SLM:000000007,Class,Sphingomyelin (iso-d17:1(4E)),SM(iso-d17:1(4E)),N-acyl-15-methylhexadecasphing-4-enine-1-phosp...,SLM:000001000,NaN,NaN,CC(C)CCCCCCCCC\C=C\[C@@H](O)[C@H](COP([O-])(=O...,InChI=none,...,NaN,NaN,NaN,NaN,NaN,70775,NaN,NaN,MNXM97113,14685263 | 21926990 | 9603947
4,SLM:000000035,Isomeric subspecies,sphinganine,NaN,NaN,SLM:000390097,NaN,NaN,CCCCCCCCCCCCCCC[C@@H](O)[C@@H]([NH3+])CO,InChI=1S/C18H39NO2/c1-2-3-4-5-6-7-8-9-10-11-12...,...,308.313535,319.331905,300.290803,336.267481,360.311932,57817,LMSP01020001,HMDB00269,MNXM302,10652340 | 10702247 | 10751414 | 10802064 | 10...


In [3]:
def clean_abbreviation(abbrev):
    if pd.isna(abbrev):
        return abbrev

    # Rule 1: Return NaN if no '(' or ')'
    if '(' not in abbrev or ')' not in abbrev:
        return np.nan

    # Rule 2: Repeated substitution until pattern is gone
    pattern = r'\((d|iso-|t)'
    while re.search(pattern, abbrev):
        abbrev = re.sub(pattern, '(', abbrev)

    # Rule 3: Remove nested parentheses inside the first matching pair
    match = re.search(r'\((.*)\)', abbrev)
    if match:
        content = match.group(1)
        cleaned_content = re.sub(r'\([^()]*\)', '', content)
        abbrev = re.sub(r'\(.*\)', f'({cleaned_content})', abbrev)

    # Rule 4: Add space before first '(' if not already there
    abbrev = re.sub(r'(?<!\s)\(', ' (', abbrev, count=1)

    # Rule 5: Remove all parentheses
    abbrev = abbrev.replace('(', '').replace(')', '')

    return abbrev

In [4]:
swiss_lipids = swiss_lipids.rename(columns={'InChI key (pH7.3)': 'inchikey', 'Abbreviation*': 'NAME'})
swiss_lipids['NAME'] = swiss_lipids['NAME'].apply(clean_abbreviation)
swiss_lipids = swiss_lipids[swiss_lipids['NAME'].notna()]
swiss_lipids = swiss_lipids.dropna(subset=['inchikey'])
swiss_lipids['INCHIKEY'] = swiss_lipids['inchikey'].apply(lambda x: x[9:23])
swiss_lipids = swiss_lipids[['NAME', 'INCHIKEY']]

In [5]:
swiss_lipids

,NAME,INCHIKEY
17,NAE 18:1,BOWVQLFMWHZBEF
18,NAE 16:0,HXYVTAGFYLMHSO
19,Cer 17:1/22:0,XMCZTIGIXKXPGG
20,Cer 17:1/21:0,XTOGDASSFFGCNZ
21,Cer 17:1/21:0-2OH,QEHCU-UGRYJNRP
...,...,...
779136,MSGG 18:0/14:1,UNTUAOPYHICUAY
779137,MSGG 18:0/32:5,AVYFDZVTTCCVMW
779138,MSGG 18:0/34:5,NSODXZFIODTENP
779139,MSGG 18:0/36:5,ZMPSYRJJHIFJNT


### MS-DIAL

In [6]:
ms_dial_lipid = pd.read_csv('MS-DIAL/msdial_ms2.csv')
ms_dial_lipid.head(5)

,NAME,PRECURSORMZ,PRECURSORTYPE,SMILES,INCHIKEY,FORMULA,RETENTIONTIME,IONMODE,COMPOUNDCLASS,Comment,Num Peaks,MS2
0,CAR 4:0,232.15433,[M]+,CCCC(=O)OC(CC(O)=O)C[N+](C)(C)C,QWYFHHGCZUCMBN-UHFFFAOYNA-O,C11H22NO4,0.51,Positive,CAR,NaN,2,"[[85.02841, 800], [232.1543, 999]]"
1,CAR 5:0,246.16998,[M]+,CCCCC(=O)OC(CC(O)=O)C[N+](C)(C)C,VSNFQQXVMPSASB-UHFFFAOYNA-O,C12H24NO4,0.68,Positive,CAR,NaN,2,"[[85.02841, 800], [246.17, 999]]"
2,CAR 6:0,260.18563,[M]+,CCCCCC(=O)OC(CC(O)=O)C[N+](C)(C)C,VVPRQWTYSNDTEA-UHFFFAOYNA-O,C13H26NO4,0.86,Positive,CAR,NaN,2,"[[85.02841, 800], [260.1856, 999]]"
3,CAR 6:1,258.16998,[M]+,C\C=C\CCC(=O)OC(CC(O)=O)C[N+](C)(C)C,ILBCEMJKBZEGEY-AATRIKPKNA-O,C13H24NO4,0.54,Positive,CAR,NaN,2,"[[85.02841, 800], [258.17, 999]]"
4,CAR 6:2,256.15433,[M]+,C\C=C\C=C\C(=O)OC(CC(O)=O)C[N+](C)(C)C,ZNVXKVUKEKVBRW-BSWSSELBNA-O,C13H22NO4,0.49,Positive,CAR,NaN,2,"[[85.02841, 800], [256.1543, 999]]"


In [7]:
ms_dial_lipid['INCHIKEY'] = ms_dial_lipid['INCHIKEY'].str[:14]
ms_dial_lipid = ms_dial_lipid[['NAME', 'INCHIKEY']]
unique_inchikeys = ms_dial_lipid['INCHIKEY'].value_counts()

In [8]:
ms_dial_lipid

,NAME,INCHIKEY
0,CAR 4:0,QWYFHHGCZUCMBN
1,CAR 5:0,VSNFQQXVMPSASB
2,CAR 6:0,VVPRQWTYSNDTEA
3,CAR 6:1,ILBCEMJKBZEGEY
4,CAR 6:2,ZNVXKVUKEKVBRW
...,...,...
911870,TG 24:6_24:6_26:4,SBHKPFZSEAIYRQ
911871,TG 24:6_24:6_26:5,RZESYAGEIOFUND
911872,TG 24:6_24:6_26:6,GGOIWQSKSHLAEF
911873,TG 24:6_24:6_26:7,GZBRUNULWHILAO


### Merge

In [9]:
print(ms_dial_lipid.shape, swiss_lipids.shape)

(911875, 2) (775982, 2)


In [10]:
lipid_list = pd.concat([swiss_lipids, ms_dial_lipid], ignore_index=True)
lipid_list = lipid_list[~lipid_list['INCHIKEY'].duplicated(keep=False)]

In [11]:
lipid_list

,NAME,INCHIKEY
0,NAE 18:1,BOWVQLFMWHZBEF
1,NAE 16:0,HXYVTAGFYLMHSO
2,Cer 17:1/22:0,XMCZTIGIXKXPGG
3,Cer 17:1/21:0,XTOGDASSFFGCNZ
4,Cer 17:1/21:0-2OH,QEHCU-UGRYJNRP
...,...,...
1388691,SHexCer 30:1;2O/40:1,GHGPFHWUUWDGQK
1388692,SHexCer 30:1;2O/40:2,RSWVBJQVSFXCCW
1388693,SHexCer 30:1;2O/42:0,JZZQMRYDYUGIPB
1388694,SHexCer 30:1;2O/42:1,JZMNMKXSZMSZSG


# MS2 Datasets only keep lipid

In [12]:
def only_lipid(df_name, lipid_df):
    """
    Updates the 'NAME' column in df_name based on matching 'INCHIKEY MAIN BLOCK'
    with the 'INCHIKEY' column in lipid_df. If a match is found, df_name['NAME'] is
    replaced with the corresponding lipid_df['NAME'] value. Rows with no match are dropped.

    Parameters:
        df_name (pd.DataFrame): DataFrame with at least the columns 'INCHIKEY MAIN BLOCK' and 'NAME'.
        lipid_df (pd.DataFrame): DataFrame with at least the columns 'INCHIKEY' and 'NAME'.

    Returns:
        pd.DataFrame: The updated DataFrame containing only rows where a match was found.
    """
    # Determine which key column exists in df_name
    if 'INCHIKEY MAIN BLOCK' in df_name.columns:
        key_col = 'INCHIKEY MAIN BLOCK'
    elif 'inchikey' in df_name.columns:
        key_col = 'inchikey'
    else:
        raise KeyError("No matching key column found in df_name: 'INCHIKEY MAIN BLOCK' or 'inchikey'")
    
    # Create a mapping from lipid_df's INCHIKEY to its NAME
    mapping = lipid_df.set_index('INCHIKEY')['NAME']
    
    # Filter df_name to keep only rows where the key column exists in the mapping
    df_name = df_name[df_name[key_col].isin(mapping.index)].copy()
    
    # Replace the 'NAME' column in df_name using the mapping
    df_name.loc[:, 'NAME'] = df_name[key_col].map(mapping)
    
    # Drop the 'Unnamed: 0' column if it exists
    df_name = df_name.drop(columns=['Unnamed: 0'], errors='ignore')
    
    # Reorder the DataFrame so that 'NAME' is the first column
    cols = list(df_name.columns)
    cols.remove('NAME')
    df_name = df_name[['NAME'] + cols]
    
    return df_name

In [13]:
# Load the original data
gnps_df = pd.read_csv('GNPS/GNPS.csv')
massbank_df = pd.read_csv('MassBank/MASSBANK.csv')
mona_df = pd.read_csv('MONA/MONA.csv')
massspecgym_df = pd.read_csv('MassSpecGym/MassSpecGym.csv')

# Filter only lipid entries
gnps_lipid = only_lipid(gnps_df, lipid_list)
massbank_lipid = only_lipid(massbank_df, lipid_list)
mona_lipid = only_lipid(mona_df, lipid_list)
massspecgym_lipid = only_lipid(massspecgym_df, lipid_list)

# Save the filtered data to new CSV files in the same directories
gnps_lipid.to_csv('GNPS/GNPS_lipid.csv', index=False)
massbank_lipid.to_csv('MassBank/MASSBANK_lipid.csv', index=False)
mona_lipid.to_csv('MONA/MONA_lipid.csv', index=False)
massspecgym_lipid.to_csv('MassSpecGym/MassSpecGym_lipid.csv', index=False)

In [14]:
massspecgym_lipid

,NAME,identifier,smiles,inchikey,formula,precursor_formula,parent_mass,precursor_mz,adduct,instrument_type,collision_energy,fold,simulation_challenge,MS2
3315,PC 18:1/16:0,MassSpecGymID0004100,CCCCCCCCCCCCCCCC(=O)O[C@H](COC(=O)CCCCCCC/C=C\...,RRVPPYNAZJRZFR,C42H82NO8P,C42H83NO8P,759.577824,760.5851,[M+H]+,QTOF,40.0,train,True,"[[71.0736, 0.004004004004004004], [80.9731, 0...."
3316,PC 18:1/16:0,MassSpecGymID0004101,CCCCCCCCCCCCCCCC(=O)O[C@H](COC(=O)CCCCCCC/C=C\...,RRVPPYNAZJRZFR,C42H82NO8P,C42H82NNaO8P,759.577782,782.5670,[M+Na]+,QTOF,40.0,train,False,"[[71.0853, 0.042042042042042045], [81.0688, 0...."
3317,PC 18:1/16:0,MassSpecGymID0004102,CCCCCCCCCCCCCCCC(=O)O[C@H](COC(=O)CCCCCCC/C=C\...,RRVPPYNAZJRZFR,C42H82NO8P,C42H83NO8P,759.577824,760.5851,[M+H]+,QTOF,10.0,train,True,"[[86.0963, 0.015015015015015015], [98.9833, 0...."
3318,PC 18:1/16:0,MassSpecGymID0004103,CCCCCCCCCCCCCCCC(=O)O[C@H](COC(=O)CCCCCCC/C=C\...,RRVPPYNAZJRZFR,C42H82NO8P,C42H83NO8P,759.577824,760.5851,[M+H]+,QTOF,20.0,train,True,"[[71.0733, 0.003003003003003003], [86.0969, 0...."
3319,PC 18:1/16:0,MassSpecGymID0004104,CCCCCCCCCCCCCCCC(=O)O[C@H](COC(=O)CCCCCCC/C=C\...,RRVPPYNAZJRZFR,C42H82NO8P,C42H82NNaO8P,759.577782,782.5670,[M+Na]+,QTOF,20.0,train,False,"[[71.0722, 0.007007007007007007], [85.1027, 0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231075,PE 16:0/22:4,MassSpecGymID0414127,CCCCCCCCCCCCCCCC(=O)OC[C@H](COP(=O)(O)OCCN)OC(...,SQGZFCFLUVPOSZ,C43H78NO8P,C43H79NO8P,767.546724,768.5540,[M+H]+,Orbitrap,NaN,train,False,"[[90.269203, 0.01475753330925925], [91.053932,..."
231076,PE 16:0/22:4,MassSpecGymID0414128,CCCCCCCCCCCCCCCC(=O)OC[C@H](COP(=O)(O)OCCN)OC(...,SQGZFCFLUVPOSZ,C43H78NO8P,C43H79NO8P,767.546724,768.5540,[M+H]+,Orbitrap,NaN,train,False,"[[201.12558, 0.0008352653719439692], [203.2178..."
231077,PE 16:0/22:4,MassSpecGymID0414129,CCCCCCCCCCCCCCCC(=O)OC[C@H](COP(=O)(O)OCCN)OC(...,SQGZFCFLUVPOSZ,C43H78NO8P,C43H79NO8P,767.546724,768.5540,[M+H]+,Orbitrap,NaN,train,False,"[[91.054398, 0.03602636511941229], [91.135323,..."
231080,PC 20:0/24:1,MassSpecGymID0414132,CCCCCCCCCCCCCCCCCCCC(=O)OC[C@H](COP(=O)([O-])O...,JAMSDVDUWQNQFZ,C52H102NO8P,C52H103NO8P,899.734724,900.7420,[M+H]+,Orbitrap,NaN,train,False,"[[184.10527, 0.3324689845664386], [184.72757, ..."


In [3]:
# Save the filtered data to new CSV files in the same directories
gnps_lipid = pd.read_csv('GNPS/GNPS_lipid.csv')
massbank_lipid = pd.read_csv('MassBank/MASSBANK_lipid.csv')
mona_lipid = pd.read_csv('MONA/MONA_lipid.csv')
# massspecgym_lipid = pd.read_csv('MassSpecGym/MassSpecGym_lipid.csv')

# Add source column to each dataframe
gnps_lipid['Source'] = 'GNPS'
massbank_lipid['Source'] = 'MassBank'
mona_lipid['Source'] = 'MONA'

# Merge the dataframes
merged_lipids = pd.concat([gnps_lipid, massbank_lipid, mona_lipid], ignore_index=True)

In [5]:
merged_lipids.to_csv('gnps_massbank_mona.csv', index=False)